In [13]:
import os
import pandas as pd
import json
from collections import Counter
import seaborn as sns
from netpixi.integration.gt import *
from netpixi.integration.gt import Graph
from regression.integration.gt import *
import netpixi
from collections import defaultdict
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
import cpnet
from graph_tool import spectral
import json

import numpy as np
from collections import defaultdict
import itertools
g = Graph(directed=False)
files = os.listdir('./jsons')

json_files = [file for file in files if file.endswith('.json')]

In [2]:
df_combined = pd.DataFrame()

for json_file in json_files:
    data = []
    with open(f"jsons/{json_file}", 'r', encoding='utf-8') as f:
        for line in f:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError:
                pass
    df = pd.DataFrame(data)
    df_combined = pd.concat([df_combined, df], ignore_index=True)
print(len(df_combined))

13707


In [3]:
df_filtered_computers = df_combined[df_combined['categories'].apply(lambda x: 'Computers' in x)]

In [4]:
usuarios = []
ratings = []
for reviews_produto in df_combined['reviews']:
    ratings_produto = []
    for review in reviews_produto:
        if 'username' in review.keys():
            usuarios.append(review['username'])
        if 'rating' in review.keys():
            ratings_produto.append(review['rating'])
    ratings.append(sum(ratings_produto)/len(ratings_produto))

In [5]:
from collections import defaultdict

product_reviewers = defaultdict(list)

for i, row in df_combined.iterrows():
    for review in row['reviews']:
        if 'username' in review.keys() and 'rating' in review.keys():
            username = review['username']
            product_name = row['name']
            product_reviewers[product_name].append(username)

new_df = pd.DataFrame(list(product_reviewers.items()), columns=['product_name', 'usernames'])

new_df

,product_name,usernames
0,"ThinkVision T23d 22.5"" LED Monitor (HDMI, DP, ...","[Gomi, Terryc, myOwnTwoCents, Justin from Mich..."
1,"ThinkVision M14t 14"" LED Mobile Monitor with T...","[RogueStudio, Vvvsss, Anonymous, Zusan, Jenni2..."
2,"ThinkCentre Tiny-In-One 24 Gen 4 23.8"" IPS LCD...","[Robert W in TX, ucvb9, ucvb9, AlanS3, Robert ..."
3,"ThinkVision T22v-20 21.5"" VoIP LED Monitor (HD...","[MDREAM, Kent A, Norm01, Anonymous, Sam W, Ric..."
4,Legion KM300 Full-size Wired RGB Gaming Keyboa...,"[BCarter · Review provided by lenovo.com, fwi..."
...,...,...
12967,Frigidaire FDB2410HIC - Dishwasher - built-in ...,"[Lupe51, nanapattiann, Bobbi, Ruthie, kkdidit2..."
12968,"FRIGIDAIRE FFCD2418UW0A 24"" Built-In Dishwashe...","[Stealmonkey, The ok dishwasher, Randa, Dishwa..."
12969,"Frigidaire Gallery 8,000 BTU Window Air Condit...","[Review provided by Overstock.com, Jon_M, 246..."
12970,"Frigidaire® Top Freezer Refrigerator, 18 Cu. Ft.","[TheHenees, Blanca63, Large and roomy, Cougdud..."


In [6]:
from collections import defaultdict

product_reviewers = defaultdict(lambda: defaultdict(list))

for i, row in df_combined.iterrows():
    for review in row['reviews']:
        if 'username' in review.keys() and 'rating' in review.keys():
            username = review['username']
            rating_medio = review['rating']
            product_name = row['name']
            product_reviewers[product_name]['usernames'].append(username)
            product_reviewers[product_name]['ratings'].append(rating_medio)

# Convert the defaultdict to a list of dictionaries
data = [{'product_name': k, 'usernames': v['usernames'], 'ratings': v['ratings']} 
        for k, v in product_reviewers.items()]

# Create a new DataFrame
new_df = pd.DataFrame(data)

In [7]:
ratings = []
for i, row in new_df.iterrows():
    ratings.append(sum(row['ratings'])/len(row['ratings']))

In [8]:
new_df['rating_medio'] = ratings

In [14]:
produto_rating = {}

for i, row in new_df.iterrows():
    produto_rating[row['product_name']] = row['rating_medio']

In [10]:
df_sample = new_df.sample(n=2000, random_state = 42)
print(len(df_sample))

2000


In [11]:
df_sample.to_csv('sample.csv', index=False)

<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()


In [17]:
with open('produto_rating.json', 'w') as f:
    json.dump(produto_rating, f)